In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats
import scipy

In [ ]:
# The Data
data = pd.DataFrame({'x' : [6, 9, 17, 22, 7, 5, 5, 14, 9, 7, 9, 51],
                    'y' : [5, 2, 0, 0, 2, 1, 0, 0, 0, 0, 13, 0],
                    't' : [11, 11, 17, 22, 9, 6, 5, 14, 9, 7, 22, 51]})
N = 12 # Numbers of patients
# alpha <- 0
# delta <- 0

In [21]:
stats.bernoulli.rvs(p = 0.5)

0

In [ ]:

def heart(data, priors_parameters, init_chain, lambda_params, nchain) :

    chain = np.zeros((3,nchain +1)) #theta, beta, p
    z = np.zeros((12, nchain+1)) # latent variable (state of a patient : cure/ not cure)
    chain[0] = init_chain

    x = data['x']
    y = data['y']
    t = data['t']

    for t in range(nchain+1) :
        current = chain[t]
        theta = current[0]
        beta = current[1]
        p = current[2]

        # Udapte the latent variable
        for i in range(12) :
            lambda_i = stats.gamma.rvs(loc = lambda_params[0] + x[i] + y[i], scale = lambda_params[1])
            q = theta / ( theta + (1-theta)*((beta*lambda_i)**y[i])*np.exp(-beta*lambda_i)/scipy.special.factorial(y[i]))
            z[t][i] = stats.bernoulli.rvs(p = q)

        # Udapte of theta
        alpha_theta = priors_parameters[0] + np.sum(z[t])
        beta_theta = priors_parameters[1] + 12 - np.sum(z[t])
        theta = stats.beta.rvs(alpha_theta, beta_theta)

        # Udapte of beta

        # Udapte of p

        

